In [4]:
import pandas as pd
df = pd.read_csv("/Users/sai/flight-delay-analysis/data/Airline_Delay_Cause.csv")
print("Before Cleaning")
print(df.info())
print(df.head())
df_dup = df.drop_duplicates()
df_na = df.dropna()
print("After Cleaning")
print(df_na.info())
print(df_na.head())

Before Cleaning
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24451 entries, 0 to 24450
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 24451 non-null  int64  
 1   month                24451 non-null  int64  
 2   carrier              24451 non-null  object 
 3   carrier_name         24451 non-null  object 
 4   airport              24451 non-null  object 
 5   airport_name         24451 non-null  object 
 6   arr_flights          24430 non-null  float64
 7   arr_del15            24425 non-null  float64
 8   carrier_ct           24430 non-null  float64
 9   weather_ct           24430 non-null  float64
 10  nas_ct               24430 non-null  float64
 11  security_ct          24430 non-null  float64
 12  late_aircraft_ct     24430 non-null  float64
 13  arr_cancelled        24430 non-null  float64
 14  arr_diverted         24430 non-null  float64
 15  arr_delay           

In [2]:
print("Test for flight delay")

Test for flight delay
